# ROBOUST ROUTE PLANNER

### Team: SAFJ

### Members: Serif Soner Serbest - Jelena Banjac - Fatine Benhsain -  Asli Yorusun

Using the inputs given, this notebook calculates possible routes from one station to another at a given time of a given day, taking into consideration the confidence of each connection / route.

This notebook unites the information from Data Analysis and Distance Analysis notebooks. Therefore, please take a look on those if there are any sections considered uncommented.

### TABLE OF CONTENTS

#### 1. [Data](#1)

In this section we import modify the dataset 
#### 2. [Confidence Calculation](#2)
In this section we build necessary functions and dataframes to calculate confidence for a given route
#### 3. [Connection Graph](#3)
In this section we calculate the connection graph that shows the reachability of any station pairs.
#### 4. [Timetable](#4)
In this section we create timetable that provide information about departure and arrival stations and times for a given day
#### 5. [Route](#5)
In this section we calculate each possible routes by using our timetable and connection graph and provide confidence level of the route with confidence calculation

### System Parameters

We start by defining the parameters used in our study : 

In [1]:
# change from one station to another in mins
transfer_delay = 5 

# average walking speed is assumed to be 4.5 km/h
# ref: https://www.quora.com/What-is-the-average-walking-speed-of-a-human
# in minutes
max_walking_time = 5

# m/min which corresponds to 4.5 km/h
human_speed = 75 

# as meters
max_walking_distance = human_speed * max_walking_time 

### Dependencies

In [2]:
import pickle
import socket
import getpass
import os

import ast
import numpy as np
from scipy import stats
import pandas as pd

import networkx as nx
from datetime import datetime, timedelta

import math
from math import sin, cos, sqrt, atan2, radians

import random
import json
import matplotlib.patches as mpatches
from py4j.protocol import Py4JJavaError
import networkx as nx

import matplotlib.pyplot as plt
%matplotlib inline

We change the layout in order to be able to see spark dataframes:

In [3]:
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

def fix_layout(width:int=95):
    from IPython.core.display import display, HTML
    display(HTML('<style>.container { width:' + str(width) + '% !important; }</style>'))
    
fix_layout()

We set up Spark depending on whether we work locally or on the cluster :

### Spark Setup

In [4]:
username = getpass.getuser()

SPARK_LOCAL = False

# on the laptop
if not 'iccluster' in socket.gethostname():
    # set this to the base spark directory on your system
    SPARK_LOCAL = True
    
    if username == "fatine":
        spark_home = '/home/fatine/spark-2.4.1-bin-hadoop2.7'
        
        try:
            import findspark
            findspark.init(spark_home)
        except ModuleNotFoundError as e:
            print('Info: {}'.format(e))
    elif username == "soner":
        spark_home = '/home/soner/Desktop/DSLAB2019/spark-2.4.1-bin-hadoop2.7'
        
        try:
            import findspark
            findspark.init(spark_home)
        except ModuleNotFoundError as e:
            print('Info: {}'.format(e))
            
    elif username == "jelena":
        pass
        
    
        
# cluster
if username == "jbanjac":
    ROOT_PATH = "/home/jbanjac/robust-journey-planning"
    os.environ['PYSPARK_PYTHON'] = '/opt/anaconda3/bin/python'
# local
elif username == "jelena":
    ROOT_PATH = os.getcwd()
    os.environ['PYSPARK_PYTHON'] = '/home/jelena/anaconda3/bin/python'

In [5]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import unix_timestamp, udf, desc
from pyspark.sql.types import *
from pyspark.sql import Window
from pyspark.sql.types import IntegerType

from pyspark.ml.feature import VectorAssembler,Normalizer,  PCA,VectorIndexer
from pyspark.ml.clustering import KMeans, BisectingKMeans, GaussianMixture

In [6]:
if username == "jelena"  or username == "fatine":
    spark = (SparkSession \
                .builder \
                .appName('sbb-{0}'.format(getpass.getuser())) \
                .master('local[4]') \
                .config('spark.driver.memory', '10g') \
                .config('spark.executor.memory', '4g') \
                .config('spark.executor.instances', '5') \
                .config('spark.port.maxRetries', '100') \
                .getOrCreate())
    CLUSTER_URL = "hdfs://iccluster042.iccluster.epfl.ch:8020"

elif SPARK_LOCAL:
    spark = SparkSession \
                .builder \
                .master("local") \
                .appName("roboust-journey-planing") \
                .config("spark.driver.host", "localhost") \
                .getOrCreate()
    CLUSTER_URL = ""
else:
    spark = SparkSession \
                .builder \
                .master("yarn") \
                .appName('sbb-{0}'.format(getpass.getuser())) \
                .config('spark.executor.memory', '4g') \
                .config('spark.executor.instances', '5') \
                .config('spark.port.maxRetries', '100') \
                .getOrCreate()
    CLUSTER_URL = ""

In [7]:
sc = spark.sparkContext
spark

<a id='1'></a>

## DATA

In order to give the right routes and to build our confidence estimations, we need to import and use data extracted from the sbb websites.

We start by reading the csv files containing our data :

In [8]:
if SPARK_LOCAL:
    if username == "fatine":
        df_zurich_full = spark.read.csv(f"/home/fatine/Documents/Cours Semestre Printemps/Lab in DS/*/*.csv", header=True, sep=';').cache()
    elif username == "soner":
        pass
    elif username == "jelena":
        df_zurich_full = spark.read.csv(f"{CLUSTER_URL}/datasets/sbb/*/*/*.csv.bz2", sep=';', header=True).cache()
else:
    df_zurich_full = spark.read.csv(f"{CLUSTER_URL}/datasets/sbb/*/*/*.csv.bz2", sep=';', header=True).cache()

We can now inspect our data and clean it according to our needs, for example:
    - we translate the columns from German to English
    - we only keep the stations in the Zurich area
    - we drop the connections that have null times because they are not useful for us without the adequate time
    - we fix the station names that didn't get formatted correctly

In [9]:
def filtering_of_dataframe(df):
    """ Filter dataframe
    
    This method cleans the unneeded rows from the dataframe that we concluded we don't need in Dataset Analysis notebook.
    It will be both used for the whoe dataset as well as for the 1-day dataframes.
    
    Parameters
    ----------
    df: spark dataframe
        Dataframe that contains trips information from SBB
        
    Returns
    -------
    df: spark dataframe
        Dataframe that is filtered with data we need for the best route prediction
    """
    df = df.withColumnRenamed("BETRIEBSTAG", "TRIP_DATE")\
            .withColumnRenamed("FAHRT_BEZEICHNER", "TRIP_ID")\
            .withColumnRenamed("BETREIBER_ID", "OPERATOR_ID")\
            .withColumnRenamed("BETREIBER_ABK", "OPERATOR_ABK")\
            .withColumnRenamed("BETREIBER_NAME", "OPERATOR_NAME")\
            .withColumnRenamed("PRODUKT_ID", "TRANSPORT_TYPE")\
            .withColumnRenamed("LINIEN_ID", "TRAIN_ID")\
            .withColumnRenamed("LINIEN_TEXT", "TRAIN_NAME")\
            .withColumnRenamed("UMLAUF_ID", "CIRCULATING_ID")\
            .withColumnRenamed("VERKEHRSMITTEL_TEXT", "SERVICE_TYPE")\
            .withColumnRenamed("ZUSATZFAHRT_TF", "ADDITIONAL_DRIVING")\
            .withColumnRenamed("FAELLT_AUS_TF", "FAILED")\
            .withColumnRenamed("BPUIC", "STATION_ID")\
            .withColumnRenamed("HALTESTELLEN_NAME", "STATION_NAME")\
            .withColumnRenamed("ANKUNFTSZEIT", "SCHEDULE_ARRIVE_TIME")\
            .withColumnRenamed("AN_PROGNOSE", "ACTUAL_ARRIVE_TIME")\
            .withColumnRenamed("AN_PROGNOSE_STATUS", "ACTUAL_ARRIVE_TIME_STATUS")\
            .withColumnRenamed("ABFAHRTSZEIT", "SCHEDULE_DEPART_TIME")\
            .withColumnRenamed("AB_PROGNOSE", "ACTUAL_DEPART_TIME")\
            .withColumnRenamed("AB_PROGNOSE_STATUS", "ACTUAL_DEPART_TIME_STATUS")\
            .withColumnRenamed("DURCHFAHRT_TF", "PASSES_BY")
    
    
    # load Zurich stations set
    with open('distance/zurich_stations_set.pickle', 'rb') as handle:
        zurich_stations_set = pickle.load(handle)

    df = df.where(F.col("STATION_ID").isin(zurich_stations_set)).cache()
    
    # For now we drop the connections with a null time
    df = df.na.drop(subset=["SCHEDULE_ARRIVE_TIME", "SCHEDULE_DEPART_TIME"])
    
    df = df.filter("TRANSPORT_TYPE is not null")
    df = df.filter(df.ADDITIONAL_DRIVING==False)
    df = df.filter(df.FAILED==False)

    @F.udf
    def fix_station_name(station_name):
        fixed_station_name = station_name.replace("�", "ü")
        return fixed_station_name

    df = df.withColumn('STATION_NAME', fix_station_name(df.STATION_NAME))
    df = df.filter(df.PASSES_BY == False)

    df = df.filter(df.ACTUAL_ARRIVE_TIME_STATUS != "UNBEKANNT")
    df = df.filter(df.ACTUAL_DEPART_TIME_STATUS != "UNBEKANNT")

    df = df.filter("ACTUAL_ARRIVE_TIME is not null and SCHEDULE_ARRIVE_TIME is not null")
    df = df.filter("ACTUAL_ARRIVE_TIME is not null and SCHEDULE_ARRIVE_TIME is not null")
    df = df.filter("not(ACTUAL_DEPART_TIME is null and SCHEDULE_DEPART_TIME is null)")
    df = df.filter("ACTUAL_DEPART_TIME is not null and SCHEDULE_DEPART_TIME is not null")
    
    return df

In [10]:
df_zurich_full = filtering_of_dataframe(df_zurich_full)
df_zurich_full.printSchema()

root
 |-- TRIP_DATE: string (nullable = true)
 |-- TRIP_ID: string (nullable = true)
 |-- OPERATOR_ID: string (nullable = true)
 |-- OPERATOR_ABK: string (nullable = true)
 |-- OPERATOR_NAME: string (nullable = true)
 |-- TRANSPORT_TYPE: string (nullable = true)
 |-- TRAIN_ID: string (nullable = true)
 |-- TRAIN_NAME: string (nullable = true)
 |-- CIRCULATING_ID: string (nullable = true)
 |-- SERVICE_TYPE: string (nullable = true)
 |-- ADDITIONAL_DRIVING: string (nullable = true)
 |-- FAILED: string (nullable = true)
 |-- STATION_ID: string (nullable = true)
 |-- STATION_NAME: string (nullable = true)
 |-- SCHEDULE_ARRIVE_TIME: string (nullable = true)
 |-- ACTUAL_ARRIVE_TIME: string (nullable = true)
 |-- ACTUAL_ARRIVE_TIME_STATUS: string (nullable = true)
 |-- SCHEDULE_DEPART_TIME: string (nullable = true)
 |-- ACTUAL_DEPART_TIME: string (nullable = true)
 |-- ACTUAL_DEPART_TIME_STATUS: string (nullable = true)
 |-- PASSES_BY: string (nullable = true)



<a id='2'></a>

## CONFIDENCE CALCULATION

We can now proceed to compute the confidence of connections.

We start by creating some helper functions:

In [11]:
# creating helper UDF functions
delay_arrive = F.unix_timestamp('ACTUAL_ARRIVE_TIME', format='dd.MM.yyyy HH:mm:ss') - F.unix_timestamp('SCHEDULE_ARRIVE_TIME', format='dd.MM.yyyy HH:mm')
delay_depart = F.unix_timestamp('ACTUAL_DEPART_TIME', format='dd.MM.yyyy HH:mm:ss') - F.unix_timestamp('SCHEDULE_DEPART_TIME', format='dd.MM.yyyy HH:mm')

@F.udf
def convert_to_min(delay_sec):
    minutes = math.ceil(delay_sec/60)
    return minutes

@F.udf
def convert_to_weekday_1(date):
    return str(datetime.strptime(date, '%d.%m.%Y').strftime('%w'))

@F.udf
def convert_to_weekday_2(date):
    return str(datetime.strptime(date, '%d.%m.%Y %H:%M:%S').strftime('%w'))

@F.udf
def convert_to_hour(date):
    return str(datetime.strptime(date, '%d.%m.%Y %H:%M:%S').hour)

@F.udf
def convert_to_month_1(date):
    return str(datetime.strptime(date, '%d.%m.%Y').month)

@F.udf
def convert_to_month_2(date):
    return str(datetime.strptime(date, '%d.%m.%Y %H:%M:%S').month)


Using our defined methods, we can now proceed to add the delays to our dataframe through several columns for the minutes, weekday, month and hour.

In [12]:
# creating additional columns we need to store delay values
df_zurich_full = df_zurich_full.withColumn("delay_arrive", delay_arrive).cache()
df_zurich_full = df_zurich_full.withColumn("delay_depart", delay_depart)
df_zurich_full = df_zurich_full.withColumn('delay_arrive', convert_to_min(df_zurich_full.delay_arrive))
df_zurich_full = df_zurich_full.withColumn('delay_depart', convert_to_min(df_zurich_full.delay_depart))
df_zurich_full = df_zurich_full.withColumn("TRIP_DATE_month",convert_to_month_1(df_zurich_full['TRIP_DATE']))\
                               .withColumn('TRIP_DATE_week_day', convert_to_weekday_1(df_zurich_full['TRIP_DATE']))
    
df_zurich_full = df_zurich_full.withColumn("ACTUAL_ARRIVE_TIME_month", convert_to_month_2(df_zurich_full['ACTUAL_ARRIVE_TIME']))\
                               .withColumn('ACTUAL_ARRIVE_TIME_week_day', convert_to_weekday_2(df_zurich_full['ACTUAL_ARRIVE_TIME']))\
                               .withColumn("ACTUAL_ARRIVE_TIME_hour", convert_to_hour(df_zurich_full['ACTUAL_ARRIVE_TIME']))

df_zurich_full = df_zurich_full.withColumn("ACTUAL_DEPART_TIME_month", convert_to_month_2(df_zurich_full['ACTUAL_DEPART_TIME']))\
                               .withColumn('ACTUAL_DEPART_TIME_week_day', convert_to_weekday_2(df_zurich_full['ACTUAL_DEPART_TIME']))\
                               .withColumn("ACTUAL_DEPART_TIME_hour", convert_to_hour(df_zurich_full['ACTUAL_DEPART_TIME']))

In the Data Analysis notebook, we have previously defined some clusters for different attributes and saved them in json files for reuse, to avoid recomputing them each time and to use them here too. So now, we can read them from the respective files : Each one is a json containing the cluster numbers and the different entries for each cluster.

In [13]:
def get_cluster_num(s, clusters):
    """ Get the cluster number of the element
    
    This method locates in which cluster the element belongs. E.g. service_type 85:11 is cluster 1, etc.
    
    Parameters
    ----------
    s: string
        Element for which we want to get the cluster number
    clusters: dict
        Dictionary of cluster keys and element values. E.g. {'1': ['85:11', '85:30', ...], '2': [...], ...}
        
    Returns
    -------
    k: string
        The number of the cluster
    """
    for k, v in clusters.items():
        if s in clusters[k]:
            return k
    return None

def read_clusters(file_name):
    """ Read clusters from the file (they were calculated in Dataset Analysis notebook)
    
    Parameters
    ----------
    file_name: string
        File name where dictionary of clusters is stored
        
    Returns
    -------
    clusters: dict
        Clusters dictionary
    """
    with open(f'clusters/{file_name}', 'r') as f:
        clusters = json.load(f)
    return clusters

cluster_SERVICE_TYPE = read_clusters('SERVICE_TYPE.json')
cluster_OPERATOR_ID = read_clusters('OPERATOR_ID.json')
cluster_STATION_ID = read_clusters('STATION_ID.json')

cluster_ACTUAL_ARRIVE_TIME_month = read_clusters('ACTUAL_ARRIVE_TIME_month.json')
cluster_ACTUAL_ARRIVE_TIME_hour = read_clusters('ACTUAL_ARRIVE_TIME_hour.json')
cluster_ACTUAL_ARRIVE_TIME_week_day = read_clusters('ACTUAL_ARRIVE_TIME_week_day.json')

cluster_ACTUAL_DEPART_TIME_month = read_clusters('ACTUAL_DEPART_TIME_month.json')
cluster_ACTUAL_DEPART_TIME_hour = read_clusters('ACTUAL_DEPART_TIME_hour.json')
cluster_ACTUAL_DEPART_TIME_week_day = read_clusters('ACTUAL_DEPART_TIME_week_day.json')

We can now use these clusters to compute our probabilities.


In [14]:
def arrive_distribution(arrive = df_zurich_full, date=None, time=None, trip_id=None, service_type=None, operator_id=None, transport_type=None, station_id=None):
    """ Get arrival delay probability and arrival delay distribution coefficients
    
    This method calculates the arrival delay probability and it calculates the exponential distribution coefficients
    
    Parameters
    ----------
    arrive: spark dataframe
        As default we get the full SBB dataset to calculate the probability
    date: string
        Date in format %d.%m.%Y
    time: string
        Time in format %H:%M
    trip_id: string
        Id of the trip
    service_type: string
        Type of service of the trip
    operator_id: string
        Operator id of the trip
    transport_type: string
        Transport type of the trip
    station_id: string
        Station ID
        
    Returns
    -------
    arrive_delay_distribution: tuple
        Coefficients of the exponential distribution
    arrive_delay_probability: float
        Probability of this arrival setting to be delayed
    """
    
    # extract date information
    if date:
        week_day = str(datetime.strptime(date, '%d.%m.%Y').strftime('%w'))
        month = str(datetime.strptime(date, '%d.%m.%Y').month)
    if time:
        hour = str(datetime.strptime(time, '%H:%M').hour)

    if trip_id:
        arrive = arrive.filter(arrive.TRIP_ID==trip_id).cache()
    if transport_type:
        arrive = arrive.filter(arrive.TRANSPORT_TYPE==transport_type).cache()
    
    oi_cn = get_cluster_num(operator_id, cluster_OPERATOR_ID)
    if oi_cn: arrive = arrive.where(arrive.OPERATOR_ID.isin(cluster_OPERATOR_ID[oi_cn])).cache()

    st_cn = get_cluster_num(service_type, cluster_SERVICE_TYPE)
    if st_cn: arrive = arrive.where(arrive.SERVICE_TYPE.isin(cluster_SERVICE_TYPE[st_cn])).cache()

        
    si_cn = get_cluster_num(station_id, cluster_STATION_ID)
    if si_cn: arrive = arrive.where(arrive.STATION_ID.isin(cluster_STATION_ID[si_cn])).cache()
    
    aat_m_cn = get_cluster_num(month, cluster_ACTUAL_ARRIVE_TIME_month)
    if aat_m_cn: arrive = arrive.where(arrive.ACTUAL_ARRIVE_TIME_month.isin(cluster_ACTUAL_ARRIVE_TIME_month[aat_m_cn])).cache()
        
    aat_h_cn = get_cluster_num(hour, cluster_ACTUAL_ARRIVE_TIME_hour)
    if aat_h_cn: arrive = arrive.where(arrive.ACTUAL_ARRIVE_TIME_hour.isin(cluster_ACTUAL_ARRIVE_TIME_hour[aat_h_cn])).cache()
        
    aat_wd_cn = get_cluster_num(week_day, cluster_ACTUAL_ARRIVE_TIME_week_day)
    if aat_wd_cn: arrive = arrive.where(arrive.ACTUAL_ARRIVE_TIME_week_day.isin(cluster_ACTUAL_ARRIVE_TIME_week_day[aat_wd_cn])).cache()

    # sum of all arrival trips
    sum_arrive_trips = arrive.count()

    # sum of all delayed trips
    arrive = arrive.filter('delay_arrive > 0')
    sum_delay_arrive_trips = arrive.count()

    # get probabilities of arrival delay
    arrive_delay_probability = sum_delay_arrive_trips/sum_arrive_trips
    data = arrive.select('delay_arrive').toPandas()
    arrive_delay_distribution = stats.expon.fit(np.array(list(map(int, data['delay_arrive'].values))), floc=0, scale=1)

    return arrive_delay_distribution, arrive_delay_probability

### Probability of catching the next trip

Here we are calculating the probability of catching the next trip. We know the distribution of arrival delays, as well as the time difference between this and the next trip that make a connection. Also, we use the probability of arrival delay to be delayed. After the exploring following papers: 

- [Stochastic Modelling of Train Delays and Delay Propagation in Stations](https://repository.tudelft.nl/islandora/object/uuid:caa72522-26b1-4088-afc0-59c6e5c346f6/datastream/OBJ/download)
- [Adi Botea, Stefano Braghin, "Contingent versus Deterministic Plans in Multi-Modal Journey Planning". ICAPS 2015: 268-272](https://dl.acm.org/citation.cfm?id=3038699)
- [Mathematical modeling and methods for rescheduling
trains under disrupted operations](https://tel.archives-ouvertes.fr/tel-00453640/document)
- [Adi Botea, Stefano Braghin, "Contingent versus Deterministic Plans in Multi-Modal Journey Planning". ICAPS 2015: 268-272.](https://dl.acm.org/citation.cfm?id=3038699)
- [Adi Botea, Evdokia Nikolova, Michele Berlingerio, "Multi-Modal Journey Planning in the Presence of Uncertainty". ICAPS 2013.](https://www.aaai.org/ocs/index.php/ICAPS/ICAPS13/paper/view/6023)

we decided to use the next way of calculating the probability:

In [15]:
def calculate_probability_to_catch(arrive_delay_distribution, timediff, arrive_delay_probability):
    """ Calculating the probability of catching the next trip
    
    Parameters
    ----------
    arrive_delay_distribution: tuple
        Coefficients of arrive delay exponential distribution
    timediff: float
        Time difference between this and next trip
    arrive_delay_probability: float
        Probability of arrival delay
    
    Returns
    -------
    p: float
        Probability of catching the next trip
    """
    timediff = int(timediff)
    quantile = np.arange (0, timediff + 1, 1) 
    R = stats.expon.cdf(quantile, loc = 0, scale = arrive_delay_distribution[1])
    
    if arrive_delay_probability == 0:
        p = 1
    else:
        p = (1 - arrive_delay_probability) + arrive_delay_probability * R[timediff]

    return p

In [16]:
def calculate_confidence(connections_info):
    """ Calculate the confidence of the whole route
    
    Parameters
    ----------
    connections_info: dict
        Information of how one found route is connected
        
    Returns
    -------
    p: float
        Confidence of this connection to succeed
    """
    p = 1.0
    
    for idx in range(len(connections_info)-1):
        
        arrive_delay_distribution, arrive_delay_probability = None, None

        if connections_info[idx]['transport_type'] != 'walk' and connections_info[idx+1]['transport_type'] != 'walk':

            arrive_delay_distribution, arrive_delay_probability = arrive_distribution(date=connections_info[idx]['arrive_date'].strftime('%d.%m.%Y'),
                                                          time=connections_info[idx]['arrive_date'].strftime('%H:%M'),
                                                          trip_id=connections_info[idx]['trip_id'],
                                                          service_type=connections_info[idx]['service_type'],
                                                          operator_id=connections_info[idx]['operator_id'],
                                                          transport_type=connections_info[idx]['transport_type'],
                                                          station_id=connections_info[idx]['arrive_station_id'])

            timediff = int((connections_info[idx+1]['depart_date']- connections_info[idx]['arrive_date']).seconds)/60

            p *= calculate_probability_to_catch(arrive_delay_distribution, timediff, arrive_delay_probability)
            
        elif connections_info[idx]['transport_type'] == 'walk' and connections_info[idx+1]['transport_type'] != 'walk':
            if idx == 0:
                arrive_delay_probability = 0
                arrive_delay_distribution = stats.expon.fit(np.zeros(1000), floc=0, scale=1)
            
            timediff = float((connections_info[idx+1]['depart_date'] - connections_info[idx]['arrive_date']).seconds)/60
            
            p *= calculate_probability_to_catch(arrive_delay_distribution, timediff, arrive_delay_probability)
        
        elif connections_info[idx]['transport_type'] != 'walk' and connections_info[idx+1]['transport_type'] == 'walk':
            arrive_delay_distribution, arrive_delay_probability = arrive_distribution(date=connections_info[idx]['arrive_date'].strftime('%d.%m.%Y'),
                                                          time=connections_info[idx]['arrive_date'].strftime('%H:%M'),
                                                          trip_id=connections_info[idx]['trip_id'],
                                                          service_type=connections_info[idx]['service_type'],
                                                          operator_id=connections_info[idx]['operator_id'],
                                                          transport_type=connections_info[idx]['transport_type'],
                                                          station_id=connections_info[idx]['arrive_station_id'])
            timediff = float((connections_info[idx+1]['depart_date']- connections_info[idx]['arrive_date']).seconds)/60

            p *= calculate_probability_to_catch(arrive_delay_distribution, timediff, arrive_delay_probability)
        
    return p

<a id='3'></a>

# CONNECTION GRAPH

Then we need an adjacency matrix that represents all the possible connections between stations without considering the time at first.

For the step-by-step development of this section, please refer to the notebook Robust_Route_Planner_OneDay.ipynb, as well as the Distance_Analysis.ipynb

In [17]:
from pyspark.sql.types import TimestampType, IntegerType, StructType, ArrayType, StructField

In [18]:
@F.udf('float')
def calculateCrossDistance(lat1, lon1, lat2, lon2):
    """ The distance between two stations, in meters
    
    Parameters
    ----------
    lat1: float
        Latitude of 1
    lon1: float
        Longitude of 1
    lat2: float
        Latitude of 2
    lon2: float
        Longitude of 2
        
    Returns
    -------
    distance: float
        Distance between these two points
    """
    # approximate radius of earth in km
    R = 6373.0

    # latitude and longitude values of a given station in terms of radians 
    # for comparing with the Zurich HB's coordinates
    lon_1 = radians(lon1)
    lat_1 = radians(lat1)
    lon_2 = radians(lon2)
    lat_2 = radians(lat2)
    
    # calculates the distance by using Haversine formula
    dlon = lon_2 - lon_1
    dlat = lat_2 - lat_1
    a = sin(dlat / 2)**2 + cos(lat_1) * cos(lat_2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c * 1000 # as m
    
    return distance

@F.udf
def calculateDistance(latitude, longitude):
    """ The distance between a coordinate and Zurich HB, in kilometers
    
    Parameters
    ----------
    latitude: float
        Latitude of the station
    longitude: float
        Longitude of the station
        
    Returns
    -------
    distance: float
        Distance from the station to Zurich HB
    """ 
    
    # approximate radius of earth in km
    R = 6373.0
    Zurich_HB_lat = 47.378178
    Zurich_HB_lon = 8.540192
    
    # latitude and longitude values of Zurich HB in terms of radians
    lat_Zurich_HB = radians(Zurich_HB_lat)
    lon_Zurich_HB = radians(Zurich_HB_lon)

    # latitude and longitude values of a given station in terms of radians 
    # for comparing with the Zurich HB's coordinates
    lon = radians(longitude)
    lat = radians(latitude)
    
    # calculates the distance by using Haversine formula
    dlon = lon - lon_Zurich_HB
    dlat = lat - lat_Zurich_HB
    a = sin(dlat / 2)**2 + cos(lat_Zurich_HB) * cos(lat) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c # as km
    
    return distance

def read_hrdf(file_name):
    """ Read the HRDF file
    
    Parameters
    ----------
    file_name: string
        File name of BFKOORD_GEO
    
    Returns
    -------
    data: list of lists
        Data in the BFKOORD_GEO file
    """
    with open(file_name, encoding='utf-8') as f:
        lines = f.readlines()
    
    data = []
    for line in lines:
        data.append([line[:7].strip(), float(line[8:18].strip()), float(line[19:29].strip()), float(line[30:36].strip()), line[38:].strip()])
    return data


def get_distance_dictionary():
    """ Get distance dictionary
    
    Returns
    -------
    distance_dictionary: dict
        Get the dictionary with distances of stations
    """
    df = pd.DataFrame(data=read_hrdf('BFKOORD_GEO'), columns=["stop_number", "longitude", "latitude", "elevation", "stop_name"])
    # convert to Spark DF
    df_geo = spark.createDataFrame(df)

    # Select the stations around 10 km of Zürich HB
    df_zurich_geo = df_geo.withColumn('distance_to_Zurich', calculateDistance(df_geo.latitude,df_geo.longitude)).filter('distance_to_Zurich<=10.0')
    
    walk_from_df = df_zurich_geo.alias('walk_from_df').withColumnRenamed('longitude', 'longitude_from')\
                                                      .withColumnRenamed('latitude', 'latitude_from')\
                                                      .withColumnRenamed('stop_number', 'stop_number_from')\
                                                      .withColumnRenamed('stop_name', 'stop_name_from')\
                                                      .drop('elevation').drop('distance_to_Zurich')

    walk_to_df = df_zurich_geo.alias("walk_to_df").withColumnRenamed('longitude', 'longitude_to')\
                                                  .withColumnRenamed('latitude', 'latitude_to')\
                                                  .withColumnRenamed('stop_number', 'stop_number_to')\
                                                  .withColumnRenamed('stop_name', 'stop_name_to')\
                                                  .drop('elevation').drop('distance_to_Zurich')
    # join these two dfs
    df_joined = walk_from_df.crossJoin(walk_to_df)

    # drop the row if arrival and departure stations are the same
    mask = df_joined.stop_number_from == df_joined.stop_number_to
    df_joined = df_joined[~mask]
    df_distances = df_joined.withColumn('distance_as_meter', calculateCrossDistance(df_joined.latitude_from,df_joined.longitude_from,df_joined.latitude_to,df_joined.longitude_to))\
                                .drop('longitude_from').drop('longitude_to').drop('latitude_from').drop('latitude_to')

    
    distance_dictionary = {(row[0], row[1]):row[2] for row in df_distances.select('stop_number_from', 'stop_number_to', 'distance_as_meter').collect()}
    
    return distance_dictionary

In [19]:
def get_transportation_matrix(df_trips, station_index_to_id, station_id_to_index):
    """ Get transportation matrix
    
    Parameters
    ----------
    df_trips: spark dataframe
        Dataframe containing trips info
    station_index_to_id: dict
        Dictionary to help convert station index to id
    station_id_to_index: dict
        Dictionary to help convert station id to index
    """

    # window by trip id and sort by arrival time
    trip_id_window = Window.partitionBy('TRIP_ID').orderBy(F.asc('SCHEDULE_ARRIVE_TIME'))
    
    # keep the order of stations in the trip
    df_trips = df_trips.withColumn('TRIP_ORDER', F.rank().over(trip_id_window)).cache()
                                   
    # define udf and struct type to get all connected station pairs
    schema = StructType([
        StructField("DEPART", ArrayType(IntegerType()), False),
        StructField("ARRIVE", ArrayType(IntegerType()), False)
    ])

    # iterate over each trip and connect every stations in the trip
    def return_all_connected_station_pairs(station_indices):
        depart = []
        arrive = []
        for i in range(len(station_indices)):
            for j in range(i+1, len(station_indices)):
                depart.append(station_indices[i])
                arrive.append(station_indices[j])
        return [depart, arrive]

    return_all_connected_station_pairs_udf = F.udf(return_all_connected_station_pairs, schema)

    # apply udf to df_trips
    df_connections = df_trips.groupBy('TRIP_ID')\
                             .agg(F.collect_list('STATION_INDEX').alias('STATION_INDEX'))\
                             .withColumn('STATION_INDEX', return_all_connected_station_pairs_udf('STATION_INDEX'))\
                             .select("TRIP_ID", "STATION_INDEX.DEPART", "STATION_INDEX.ARRIVE")
                                   
    transportation_matrix = np.zeros((len(station_index_to_id),len(station_index_to_id)))

    # collect all pairs as (depart, arrive)
    depart = df_connections.select('DEPART').rdd.flatMap(lambda x: x).collect()
    arrive = df_connections.select('ARRIVE').rdd.flatMap(lambda x: x).collect()

    # fill conection matrix with pairs
    depart_list = [item for trip in depart for item in trip]
    arrive_list = [item for trip in arrive for item in trip]

    for i in range(len(depart_list)):
        transportation_matrix[depart_list[i],arrive_list[i]] = 1
                       
    ### WALK MATRIX ###
    distance_dictionary  = get_distance_dictionary()
        
    transfer_delay = 5 # change from one station to another in mins

    # average walking speed is assumed to be 4.5 km/h
    # ref: https://www.quora.com/What-is-the-average-walking-speed-of-a-human
    max_walking_time = 5 # min
    human_speed = 75 # m/min which corresponds to 4.5 km/h
    max_walking_distance = human_speed * max_walking_time # as meters
    
    walk_matrix = np.zeros((len(station_index_to_id), len(station_index_to_id)))


    for station_from, station_to in distance_dictionary:
        # check if stations are exist in this day
        if str(station_from) in  station_id_to_index and str(station_to) in station_id_to_index:
            # check if the distance is acceptable
            if distance_dictionary[(station_from, station_to)] <= max_walking_distance:
                walk_matrix[station_id_to_index[str(station_from)], station_id_to_index[str(station_to)]] = 1
    
    return transportation_matrix, walk_matrix

In [20]:
def get_connection_graph(df_trips, transportation_matrix, walk_matrix):
    """ Get connections graph
    
    df_trips: spark dataframe
        Dataframe of trips
    transportation_matrix: dict
        Transportation matrinx containing, same as in explained in other notebook
    walk_matrix: dict
        Walk matrix containg the info if there is a walk between the stations
        
    Returns
    -------
    connection_graph: nx.DiGraph
        Connections graph
    """
    # merged conenction_matrix and wal_matrix into a one single adjcency matrix of all possible paths
    connection_matrix = np.logical_or(transportation_matrix, walk_matrix).astype(int)
    connection_graph = nx.DiGraph(connection_matrix)
    
    return connection_graph

<a id='4'></a>

# TIMETABLE

The matrix only contains 0 and 1 when a connection is possible at any given time. But we also need to know the times of the conneciton in addition to suggest to our user the best possible route. For that we form a timetable using the dataset already provided by doing a self join on the trip_id :

In [21]:
def get_timetable_for_day(df_trips):     
    """ Get the timetable of the day
    
    Parameters
    ----------
    df_trips: spark dataframe
        Dataframe containing trips
    
    Returns
    -------
    timetable: spark dataframe
        Contains timetable information
    """
    # create timetable of every possible connection by merging df_depart and df_arrive
    df_depart = df_trips.withColumnRenamed('STATION_ID', 'departure_station_id')\
                        .withColumnRenamed('STATION_NAME', 'departure_station_name')\
                        .withColumnRenamed('TRIP_ORDER', 'departure_trip_order')\
                        .withColumnRenamed('STATION_INDEX', 'departure_station_index')\
                        .drop('SCHEDULE_ARRIVE_TIME')

    df_arrive = df_trips.withColumnRenamed('STATION_ID', 'arrival_station_id')\
                        .withColumnRenamed('STATION_NAME', 'arrival_station_name')\
                        .withColumnRenamed('TRIP_ORDER', 'arrival_trip_order')\
                        .withColumnRenamed('STATION_INDEX', 'arrival_station_index')\
                        .drop('SCHEDULE_DEPART_TIME').drop('type', 'OPERATOR_ID', 'SERVICE_TYPE', 'TRANSPORT_TYPE')


    timetable = df_depart.join(df_arrive, on=['TRIP_ID'], how='left_outer').drop('departure_trip_order').drop('arrival_trip_order')

    # drop columns with the same departure and arrival stations
    mask = timetable.departure_station_name == timetable.arrival_station_name
    timetable = timetable[~mask]

    # time reverse connections
    mask = timetable.SCHEDULE_DEPART_TIME > timetable.SCHEDULE_ARRIVE_TIME
    timetable = timetable[~mask]
    
    return timetable

<a id='5'></a>

# ROUTE

We can now use all the functions previously defined to give our user the best possible routes : considering on the one hand the duration and on the other hand the probability of catching the connection :

In [22]:
# load location dictionary
with open('distance/location_dictionary.pickle', 'rb') as handle:
    location_dictionary = pickle.load(handle)

We start by removing the paths that are walkable and keeping only the direct one (if many paths involve only walking):

In [23]:
def filter_paths(connection_graph, departure_station_index, arrival_station_index, transportation_matrix, walk_matrix):
    """ Filter paths we need
    
    Parameters
    ----------
    connection_graph: nx.DiGraph  
        Connection graph
    departure_station_index: int
        Index of the departure station
    arrival_station_index: int
        Index of the arrival station
    transportation_matrix: dict
        Transportation matrix
    walk_matrix: dict
        Walk matrix
    
    Returns
    -------
    filtered_paths: list
        Filtered paths
    """
    filtered_paths = []
    for path in nx.all_simple_paths(connection_graph, departure_station_index, arrival_station_index, 2):
        if len(path) == 3:
            # check if both connection is by walk or not
            depart_index, arrive_index = path[0], path[1]
            if walk_matrix[depart_index, arrive_index] == 1:
                depart_index, arrive_index = path[1], path[2]
                # if both connection is by walk, dont add to the paths
                if walk_matrix[depart_index, arrive_index] == 1:
                    continue
        
        filtered_paths.append(path)
            
    return filtered_paths, walk_matrix

We can now calculate the routes using networkx library and them check the times using the timetable. Finally we sort the routes found by probability and duration and show the shortest ones first :

In [27]:
def calculate_routes(departure_station, arrival_station, date, hour):  
    """ Calculate possible routes based on user input
    
    Parameters
    ----------
    departure_station, 
        Departure station ID
    arrival_station: string
        Arrival station ID
    date: string
        Date in format %Y-%m-%d
    hour: string
        Hour in format %HH:%MM:%SS
    
    Returns
    -------
    all_possible_connections: list of dicts
        List of all the possible connections
    all_probabilities: list
        List of probabilities of each of the connections
    """
    
    departure_day = datetime.strptime(date, '%Y-%m-%d')

    if SPARK_LOCAL:
        if username == "fatine":
            df_zurich = spark.read.csv(f"/home/fatine/Documents/Cours Semestre Printemps/Lab in DS/{departure_day.strftime('%Y-%m')}/{departure_day.strftime('%Y-%m-%d')}istdaten.csv", header=True, sep=';').cache()

        elif username == "soner":
            df_zurich = spark.read.csv(f"/home/soner/Desktop/DSLAB2019/Project/{departure_day.strftime('%Y-%m')}/{departure_day.strftime('%Y-%m-%d')}istdaten.csv", header=True, sep=';').cache()
        elif username == "jelena":
            df_zurich = spark.read.csv(f"{CLUSTER_URL}/datasets/sbb/{str(departure_day.year)}/{str(departure_day.month).zfill(2)}/{departure_day.strftime('%Y-%m-%d')}istdaten.csv.bz2", sep=';', header=True).cache()
    else:
        df_zurich = spark.read.csv(f"{CLUSTER_URL}/datasets/sbb/{str(departure_day.year)}/{str(departure_day.month).zfill(2)}/{departure_day.strftime('%Y-%m-%d')}istdaten.csv.bz2", sep=';', header=True).cache()
    
    df_zurich = filtering_of_dataframe(df_zurich)
                                   
    # create mappings f of stations from id to index and index to id
    station_index_to_id = list(df_zurich.select('STATION_ID').distinct().toPandas()['STATION_ID'])
                                   
    station_id_to_index = {}
    for index, station_id in enumerate(station_index_to_id):
        station_id_to_index[station_id] = index
             
    # helper methods for the spark dataframe
    return_index = udf(lambda station_id: station_id_to_index[station_id], IntegerType())
    return_datetime = udf(lambda date: datetime.strptime(date, "%d.%m.%Y %H:%M"), TimestampType())
               
    df_trips = df_zurich.filter(F.col('ADDITIONAL_DRIVING')=='false')\
                        .filter(F.col('PASSES_BY')=='false')\
                        .select(F.col('OPERATOR_ID'), F.col('SERVICE_TYPE'),F.col('TRIP_ID'),
                                F.col('TRANSPORT_TYPE'),
                                F.col('STATION_ID'),
                                return_index('STATION_ID').astype('int').alias('STATION_INDEX'),
                                F.col('STATION_NAME'),
                                return_datetime(F.col('SCHEDULE_ARRIVE_TIME')).alias('SCHEDULE_ARRIVE_TIME'),
                                return_datetime(F.col('SCHEDULE_DEPART_TIME')).alias('SCHEDULE_DEPART_TIME')).cache()                               
                                   
    # get timetable dataframe                               
    timetable = get_timetable_for_day(df_trips)
    # get transportation and walk matrices
    transportation_matrix, walk_matrix = get_transportation_matrix(df_trips, station_index_to_id, station_id_to_index)
    # get connection graph
    connection_graph = get_connection_graph(df_trips, transportation_matrix, walk_matrix)
    # get distance dictionary
    distance_dictionary = get_distance_dictionary()                               
    
    # find indices of stations in the graph
    departure_station_index = station_id_to_index[departure_station]
    arrival_station_index = station_id_to_index[arrival_station]

    all_possible_connections = []
    all_probabilities = []
                            
    # find all simple paths in the graph
    try:
        paths, walk_matrix = filter_paths(connection_graph, departure_station_index, arrival_station_index, transportation_matrix, walk_matrix)
    except nx.exception.NodeNotFound:
        return None, None
    
    full_date = date + ' ' + hour                               
                                   
    i = 0
    for path in paths:    
        i = i + 1
        arrival_date = datetime.strptime(full_date, '%Y-%m-%d %H:%M:%S')

        connections_info = []

        route_failed = False
        for depart_index, arrive_index in zip(path[:-1], path[1:]):

            # check if the path is by walk or by transportation
            if walk_matrix[depart_index, arrive_index]:
                mins = distance_dictionary[(str(station_index_to_id[depart_index]), str(station_index_to_id[arrive_index]))] / human_speed

                departure_date = arrival_date 
                arrival_date = departure_date + timedelta(minutes=mins)

                connections_info.append({'depart_station_id': station_index_to_id[depart_index] , 
                                         'arrive_station_id': station_index_to_id[arrive_index]  , 
                                         'depart_date': departure_date,
                                         'arrive_date': arrival_date,
                                         'transport_type': 'walk',
                                         'trip_id': None,
                                         'operator_id':None,
                                         'service_type': None})

            else:
                # add walking time to change stations 
                arrival_date = arrival_date + timedelta(minutes=transfer_delay)
                
                # find earliest departure
                try:
                    connection = timetable.filter((F.col('departure_station_index') == depart_index) & (F.col('arrival_station_index') == arrive_index) & (F.col('SCHEDULE_DEPART_TIME') >= arrival_date)).first()
                except Py4JJavaError:
                    print('route failed')
                    route_failed = True
                    break
                #print("connection", connection)
                if connection:
                    arrival_date = connection.SCHEDULE_ARRIVE_TIME
                    mins = arrival_date - connection.SCHEDULE_DEPART_TIME
                
                    connections_info.append({'depart_station_id': connection.departure_station_id, 
                                             'arrive_station_id': connection.arrival_station_id, 
                                             'depart_date': connection.SCHEDULE_DEPART_TIME, 
                                             'arrive_date': arrival_date, 
                                             'transport_type': connection.TRANSPORT_TYPE,
                                             'trip_id': connection.TRIP_ID,
                                             'operator_id': connection.OPERATOR_ID,
                                             'service_type': connection.SERVICE_TYPE})
                else:
                    print('route failed')
                    route_failed = True
                    break

        if not route_failed:            
            p = calculate_confidence(connections_info)
        else: 
            p = 0
        
        all_possible_connections.append(connections_info)
        all_probabilities.append(p)
                                           
    # add Lon and Lat
    for ci in all_possible_connections:
        for trip in ci:
            trip['depart_station_id_LON'] =  location_dictionary[trip['depart_station_id']][0]
            trip['depart_station_id_LAT'] = location_dictionary[trip['depart_station_id']][1]
            trip['arrive_station_id_LON'] = location_dictionary[trip['arrive_station_id']][0]
            trip['arrive_station_id_LAT'] = location_dictionary[trip['arrive_station_id']][1]
     
                                   
    # Let's sort the connections first by probability                               
    max_confidence_indices = np.argsort(all_probabilities)[::-1]
    
    all_possible_connections = np.asarray(all_possible_connections)
    all_possible_connections = all_possible_connections[max_confidence_indices]
    
    all_probabilities = np.asarray(all_probabilities)
    all_probabilities = all_probabilities[max_confidence_indices]
    # Let's sort the connections now by shortest time
    durations = []
    for i in range(len(all_possible_connections)):
        d = 0
        d = (all_possible_connections[i][-1]['arrive_date'] - all_possible_connections[i][0]['depart_date']).seconds/60
        durations.append(d)

    max_confidence_indices = np.argsort(durations)

    all_possible_connections = np.asarray(all_possible_connections)
    all_possible_connections = all_possible_connections[max_confidence_indices]

    all_probabilities = np.asarray(all_probabilities)
    all_probabilities = all_probabilities[max_confidence_indices]                               

    durations= np.asarray(durations)
    durations = durations[max_confidence_indices]                               

                                        
    i = 0 
    for connections_info in all_possible_connections:    
        print('route:', i)
        p = all_probabilities[i]
        d = durations[i]
        i = i + 1
        print('----------------')
                                
        for connection in connections_info:
            print('departure: ',connection['depart_station_id'], 'arrival: ',connection['arrive_station_id'], 'departure_time: ', connection['depart_date'].strftime('%Y-%m-%d %H:%M:%S'), 'arrival_time: ', connection['arrive_date'].strftime('%Y-%m-%d %H:%M:%S'),'transport_type: ', connection['transport_type'])
        print(f"duration = {d} minutes")
        print(f"confidence = {p}")
        print('----------------')
                                       
    
    return list(all_possible_connections), list(all_probabilities), list(durations)

In [25]:
date = '2017-10-01'
hour = '15:00:00'
departure_station = 'Zürich HB'
arrival_station = 'Zürich Flughafen'
# min_confidence_level = 0.9

departure_station_id = df_zurich_full.filter("STATION_NAME = '" + departure_station + "'").first().STATION_ID
arrival_station_id = df_zurich_full.filter("STATION_NAME = '" + arrival_station + "'").first().STATION_ID
all_possible_connections, all_probabilities, durations = calculate_routes(departure_station=departure_station_id, arrival_station=arrival_station_id, date=date, hour=hour)

route: 0
----------------
departure:  8503000 arrival:  8503016 departure_time:  2017-10-01 15:39:00 arrival_time:  2017-10-01 15:49:00 transport_type:  Zug
duration = 10.0 minutes
confidence = 1.0
----------------
route: 1
----------------
departure:  8503000 arrival:  8503011 departure_time:  2017-10-01 15:17:00 arrival_time:  2017-10-01 15:19:00 transport_type:  Zug
departure:  8503011 arrival:  8503016 departure_time:  2017-10-01 15:34:00 arrival_time:  2017-10-01 15:56:00 transport_type:  Zug
duration = 39.0 minutes
confidence = 0.9999996940976795
----------------
route: 2
----------------
departure:  8503000 arrival:  8503010 departure_time:  2017-10-01 15:17:00 arrival_time:  2017-10-01 15:22:00 transport_type:  Zug
departure:  8503010 arrival:  8503016 departure_time:  2017-10-01 15:33:00 arrival_time:  2017-10-01 15:56:00 transport_type:  Zug
duration = 39.0 minutes
confidence = 0.9999916491496048
----------------
route: 3
----------------
departure:  8503000 arrival:  8503006

In [29]:
all_possible_connections, all_probabilities, durations = calculate_routes(departure_station='8503000', arrival_station='8591123', date='2017-10-01', hour='15:00:00')

route failed
route: 0
----------------
departure:  8503000 arrival:  8591067 departure_time:  2017-10-01 15:00:00 arrival_time:  2017-10-01 15:02:24 transport_type:  walk
duration = 2.4 minutes
confidence = 0.0
----------------
route: 1
----------------
departure:  8503000 arrival:  8587348 departure_time:  2017-10-01 15:00:00 arrival_time:  2017-10-01 15:01:37 transport_type:  walk
departure:  8587348 arrival:  8591123 departure_time:  2017-10-01 15:30:00 arrival_time:  2017-10-01 15:36:00 transport_type:  Tram
duration = 36.0 minutes
confidence = 1.0
----------------
route: 2
----------------
departure:  8503000 arrival:  8591174 departure_time:  2017-10-01 15:00:00 arrival_time:  2017-10-01 15:04:51 transport_type:  walk
departure:  8591174 arrival:  8591123 departure_time:  2017-10-01 15:50:00 arrival_time:  2017-10-01 15:52:00 transport_type:  Tram
duration = 52.0 minutes
confidence = 1.0
----------------


# Visualization

To run the visualization, please run the whole notebook and open the: http://0.0.0.0:5000/

In [34]:
index_html = """

<!doctype html>
<html lang="en">
<head>
  <link rel="stylesheet" href="https://cdn.rawgit.com/openlayers/openlayers.github.io/master/en/v5.3.0/css/ol.css"
    type="text/css">
  <script src="https://cdn.rawgit.com/openlayers/openlayers.github.io/master/en/v5.3.0/build/ol.js"></script>
</head>

<body>
  <div style="position: absolute; top: 10px; right: 10px; z-index: 100; background: white; padding: 8px;">
  <input type="text" placeholder="Start Station ID..." id="start_station_id" />
  <input type="text" placeholder="End Station ID..." id="end_station_id" />
  <input type="text" placeholder="yyyy-mm-dd" id="in_date" />
  <input type="text" placeholder="HH:MM:SS" id="in_time" />
  <button id="search">Show</button>
  <div id="message"></div>
</div>

  <div id="map" style="width: 100%; height: 100%"></div>

  <script type="text/javascript">
    var features = [];
    var vectorLayer = new ol.layer.Vector({
      style: function (feature, resolution) {
        return feature.get('style');
      },
    });
    var map = new ol.Map({
      target: 'map',
      layers: [
        new ol.layer.Tile({
          source: new ol.source.OSM()
        }),
      ],
      view: new ol.View({
        center: ol.proj.fromLonLat([8.540192, 47.378177]),
        zoom: 14
      })
    });
    map.addLayer(vectorLayer);

    function addLine(start, final, color) {
      var lineString = new ol.geom.LineString([start, final]);
      lineString.transform('EPSG:4326', 'EPSG:3857');
      var feature = new ol.Feature({
        geometry: lineString,
      });
      feature.setStyle(new ol.style.Style({
        stroke: new ol.style.Stroke({
          color: color,
          width: 5
        })
      }));
      features.push(feature);
      vectorLayer.setSource(
        new ol.source.Vector({
          features: features,
        })
      );
    }

    document.getElementById('search').addEventListener('click', function() {
      document.getElementById("message").innerHTML = '<br/> Loading... <br/>';
      var start_station_id = document.getElementById('start_station_id').value;
      var end_station_id = document.getElementById('end_station_id').value;
      var in_date = document.getElementById('in_date').value;
      var in_time = document.getElementById('in_time').value;

      var xhttp = new XMLHttpRequest();

      xhttp.onreadystatechange = function() {
        if (this.readyState == 4 && this.status == 200) {
          var data = JSON.parse(this.responseText);
          features = [];
          data.lines.forEach(function(line) {
            addLine(line.start, line.end, line.color);
          });
          document.getElementById("message").innerHTML = data.message;
        }
      };

      xhttp.open('GET', '/search?start=' + start_station_id + '&end=' + end_station_id + '&date=' + in_date + '&time=' + in_time, true);
      xhttp.send();
    });

  </script>
</body>
</html>

"""

In [36]:
from flask import Flask, request, jsonify


app = Flask(__name__)

def beautify_trip(trip):
    return f"take a {trip['transport_type']} from {trip['depart_station_id']} at {trip['depart_date'].strftime('%d.%m.%Y %H:%M')} to {trip['arrive_station_id']} at {trip['arrive_date'].strftime('%d.%m.%Y %H:%M')}"

def beautify_print(all_possible_connections, all_probabilities):
    colors = ["green", "red", "blue"]
    ret_val = ""
    for i, (p, ci) in enumerate(zip(all_probabilities, all_possible_connections)):
        ret_val += f"Connection {i+1} [{colors[i].upper()}] - Probability {p}:<br/>"
        for j, trip in enumerate(ci): 
            ret_val += f"{j+1}) {beautify_trip(trip)} <br/>"
        ret_val += "<br/><br/>"
    return ret_val

def make_lines(all_possible_connections):
    out_lines = []
    colors = ['#00ff00', '#ff0000', '#0000ff']
    for i, (p, ci) in enumerate(zip(all_probabilities, all_possible_connections)):
        for j, trip in enumerate(ci): 
            out_lines.append({ 'start': [ trip['depart_station_id_LON'], trip['depart_station_id_LAT']], 'end': [trip['arrive_station_id_LON'], trip['arrive_station_id_LAT']], 'color': colors[i] })
    return out_lines

@app.route('/', methods=['GET'])
def home():
    return index_html


@app.route('/search', methods=['GET'])
def search():
    print('here...')
    # Grab arguments here
    start_arg = request.args.get('start')
    end_arg = request.args.get('end')
    date_arg = request.args.get('date')
    time_arg = request.args.get('time')
    
    print(f'Ajax call executed! Start {start_arg}, end {end_arg}')
    
    # Do some processing
    #all_possible_connections, all_probabilities = find_route_with_probability(departure_station='8503000', arrival_station='8503016', date='2018-02-14', hour='00:00:00')
    all_possible_connections, all_probabilities = calculate_routes(departure_station=start_arg, arrival_station=end_arg, date=date_arg, hour=time_arg)
    all_possible_connections, all_probabilities = all_possible_connections[:3], all_probabilities[:3]
    status_message = beautify_print(all_possible_connections, all_probabilities)
    
    out_lines = make_lines(all_possible_connections)
    
    # Return data in corresponding format
    return jsonify({ 'message': status_message, 'lines': out_lines})


app.run(host='0.0.0.0', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jun/2019 18:59:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2019 18:59:59] "GET / HTTP/1.1" 200 -


here...
Ajax call executed! Start 8503000, end 8591123


[2019-06-17 19:31:01,370] ERROR in app: Exception on /search [GET]
Traceback (most recent call last):
  File "/home/jelena/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/jelena/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/jelena/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/jelena/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 35, in reraise
    raise value
  File "/home/jelena/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/jelena/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1799, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-36-0b1000b4d6